In [2]:
from util.GurobiOptimizer import GurobiOptimizer
import numpy as np

In [2]:
GurobiOptimizer = GurobiOptimizer()
solution = GurobiOptimizer.run_full_optimization()

Processing series: 100%|██████████| 153/153 [03:36<00:00,  1.42s/series]


In [ ]:
initial_demands, final_grid_demands, charging_demands, soc_t_v_s, optimal_objectives, optimization_times, mip_gaps, self_sufficiencies, pv_utilizations = solution

In [3]:
pv_utilizations = np.load('/Users/jianhern/Documents/GitHub/V2B_Optimization_with_AI_on_BSS/optimization/output/V1_WL48_PV400/pv_utilizations.npy')
self_sufficiencies = np.load('/Users/jianhern/Documents/GitHub/V2B_Optimization_with_AI_on_BSS/optimization/output/V1_WL48_PV400/self_sufficiencies.npy')



In [5]:
print(np.array(self_sufficiencies).mean(), np.array(pv_utilizations).mean())
print(np.array(self_sufficiencies).std(), np.array(pv_utilizations).std())

0.4569165745086894 0.9664461290009903
0.10280459572448873 0.030298334374897238


In [ ]:
# import matplotlib.pyplot as plt
# from pyparsing import col

# print(sum(charging_demand))
# print([round(float(i),2) for i in pv])
# print(len(pv))
# plt.figure(figsize=(12, 6))
# plt.step(range(len(initial_demand)), initial_demand , label='Initial Demand', where='post')
# plt.step(range(len(final_grid_demand)), final_grid_demand, label='Final Demand', color='black', where='post')
# plt.step(range(len(charging_demand)), [final_grid_demand[i] - charging_demand[i] for i in range(len(charging_demand))], label='Charging Demand', color='orange', where='post')
# plt.xlabel('Time Step')
# plt.ylabel('Demand (kW)')
# plt.title('Demand Comparison')
# plt.legend()
# plt.grid(True)
# plt.show()

NameError: name 'charging_demand' is not defined